<a href="https://colab.research.google.com/github/RedDawe/Colaborative_Filtering/blob/master/restaurchant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d uciml/restaurant-data-with-consumer-ratings

  0% 0.00/38.9k [00:00<?, ?B/s]
100% 38.9k/38.9k [00:00<00:00, 34.3MB/s]


In [0]:
!unzip restaurant-data-with-consumer-ratings.zip

Archive:  restaurant-data-with-consumer-ratings.zip
  inflating: README                  
  inflating: chefmozaccepts.csv      
  inflating: chefmozcuisine.csv      
  inflating: chefmozhours4.csv       
  inflating: chefmozparking.csv      
  inflating: geoplaces2.csv          
  inflating: rating_final.csv        
  inflating: usercuisine.csv         
  inflating: userpayment.csv         
  inflating: userprofile.csv         


In [0]:
import tensorflow as tf
assert tf.executing_eagerly()
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [0]:
ratings = pd.read_csv('rating_final.csv')
ratings.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [0]:
ratings = ratings.iloc[:, :3]
ratings.head()

,userID,placeID,rating
0,U1077,135085,2
1,U1077,135038,2
2,U1077,132825,2
3,U1077,135060,1
4,U1068,135104,1


In [0]:
ratings['userID'] = LabelEncoder().fit_transform(ratings['userID'])
ratings['placeID'] = LabelEncoder().fit_transform(ratings['placeID'])
ratings.head()

,userID,placeID,rating
0,76,123,2
1,76,84,2
2,76,31,2
3,76,105,1
4,67,126,1


In [0]:
print(ratings['userID'].max(), ratings['placeID'].max())

137 129


In [0]:
user_input = tf.keras.Input(shape=[1])
place_input = tf.keras.Input(shape=[1])

user = tf.keras.layers.Embedding(137+1, 10, input_length=1)(user_input)
place = tf.keras.layers.Embedding(129+1, 10, input_length=1)(place_input)

user = tf.keras.layers.Flatten()(user)
place = tf.keras.layers.Flatten()(place)

#model = tf.reduce_sum(user * place)
model = tf.keras.layers.Concatenate()(list([user, place]))
model = tf.keras.layers.Dense(1)(model)

model = tf.keras.Model([user_input, place_input], model)
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_30 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_31 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_29 (Embedding)        (None, 1, 10)        1380        input_30[0][0]                   
__________________________________________________________________________________________________
embedding_30 (Embedding)        (None, 1, 10)        1300        input_31[0][0]                   
___________________________________________________________________________________________

In [0]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [0]:
model.fit([ratings['userID'].to_numpy(), ratings['placeID'].to_numpy()], ratings['rating'].to_numpy(), epochs=100, validation_split=0.2)

Train on 928 samples, validate on 233 samples
Epoch 1/100
928/928 [==============================] - 0s 390us/sample - loss: 0.2817 - val_loss: 0.1903
Epoch 2/100
928/928 [==============================] - 0s 72us/sample - loss: 0.2805 - val_loss: 0.1974
Epoch 3/100
928/928 [==============================] - 0s 64us/sample - loss: 0.2800 - val_loss: 0.2049
Epoch 4/100
928/928 [==============================] - 0s 67us/sample - loss: 0.2782 - val_loss: 0.2113
Epoch 5/100
928/928 [==============================] - 0s 67us/sample - loss: 0.2771 - val_loss: 0.2174
Epoch 6/100
928/928 [==============================] - 0s 74us/sample - loss: 0.2764 - val_loss: 0.2227
Epoch 7/100
928/928 [==============================] - 0s 72us/sample - loss: 0.2761 - val_loss: 0.2283
Epoch 8/100
928/928 [==============================] - 0s 68us/sample - loss: 0.2755 - val_loss: 0.2320
Epoch 9/100
928/928 [==============================] - 0s 69us/sample - loss: 0.2753 - val_loss: 0.2379
Epoch 10/100
928/

Simple colaborative filtering produces comparable results to models using user nad restaurant features, without having to acces and store such information.